# 08 — Nonparametric Methods
**Author:** Ebenezer Adjartey

Covers: Mann-Whitney U, Wilcoxon signed-rank, Kruskal-Wallis, Spearman/Kendall correlations, KS test, kernel density estimation, bootstrap CI.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import (mannwhitneyu, wilcoxon, kruskal, spearmanr, kendalltau,
                          kstest, ks_2samp, rankdata)
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(42)
sns.set_theme(style='whitegrid')
print('Libraries loaded.')

## 1. Data Generation

In [ ]:
# Skewed distributions (violate normality)
group_a = np.random.exponential(scale=3, size=50)  # right-skewed
group_b = np.random.exponential(scale=4, size=50)  # different location
paired_before = np.random.gamma(3, 2, 40)
paired_after  = paired_before * 0.8 + np.random.normal(0, 0.5, 40)

g1 = np.random.exponential(2, 30)
g2 = np.random.exponential(3, 30)
g3 = np.random.exponential(5, 30)

x_rank = np.random.normal(50, 10, 60)
y_rank = x_rank * 0.7 + np.random.normal(0, 8, 60)

print('Group A mean:', group_a.mean().round(3))
print('Group B mean:', group_b.mean().round(3))

## 2. Mann-Whitney U Test (Wilcoxon Rank-Sum)

In [ ]:
# H0: group_a and group_b come from same distribution
U, p_mw = mannwhitneyu(group_a, group_b, alternative='two-sided')
print(f'Mann-Whitney U = {U:.4f}')
print(f'p-value        = {p_mw:.4f}')
print('Verdict:', 'Distributions differ' if p_mw < 0.05 else 'No significant difference')

# Effect size: rank-biserial correlation
n_a, n_b = len(group_a), len(group_b)
r_rb = 1 - 2*U/(n_a*n_b)
print(f'\nRank-biserial r = {r_rb:.4f} (|r|>0.5 = large effect)')

## 3. Wilcoxon Signed-Rank Test (Paired)

In [ ]:
# H0: median difference = 0 (paired, non-normal data)
W, p_wsr = wilcoxon(paired_before, paired_after, alternative='two-sided')
print(f'Wilcoxon W = {W:.4f}')
print(f'p-value    = {p_wsr:.4f}')
print('Verdict:', 'Significant change' if p_wsr < 0.05 else 'No significant change')

## 4. Kruskal-Wallis Test (Non-parametric ANOVA)

In [ ]:
# H0: all groups have same distribution (> 2 groups)
H, p_kw = kruskal(g1, g2, g3)
print(f'Kruskal-Wallis H = {H:.4f}')
print(f'p-value          = {p_kw:.4f}')
print('Verdict:', 'At least one group differs' if p_kw < 0.05 else 'No significant difference')

# Post-hoc: Dunn's test (pairwise)
from itertools import combinations
groups_dict = {'G1':g1,'G2':g2,'G3':g3}
print('\nPost-hoc pairwise Mann-Whitney:')
pairs = list(combinations(groups_dict.keys(), 2))
p_raw = []
for g_a, g_b in pairs:
    _, p = mannwhitneyu(groups_dict[g_a], groups_dict[g_b], alternative='two-sided')
    p_raw.append(p)

from statsmodels.stats.multitest import multipletests
_, p_adj, _, _ = multipletests(p_raw, method='holm')
for (a,b), p_r, p_a in zip(pairs, p_raw, p_adj):
    print(f'  {a} vs {b}: raw_p={p_r:.4f}, adj_p={p_a:.4f}')

## 5. Spearman and Kendall Rank Correlations

In [ ]:
rho, p_sp = spearmanr(x_rank, y_rank)
tau, p_kd = kendalltau(x_rank, y_rank)
r_pear, p_pear = stats.pearsonr(x_rank, y_rank)

print(f'Pearson  r   = {r_pear:.4f}, p = {p_pear:.4f}')
print(f'Spearman rho = {rho:.4f},    p = {p_sp:.4f}')
print(f'Kendall  tau = {tau:.4f},    p = {p_kd:.4f}')
print('\nSpearman/Kendall are robust to outliers and non-linear monotonic relationships')

## 6. Kolmogorov-Smirnov Test

In [ ]:
# One-sample KS: test if data follows a specific distribution
ks_stat, ks_p = kstest(group_a, 'expon', args=(0, group_a.mean()))
print(f'KS vs Exponential: D={ks_stat:.4f}, p={ks_p:.4f}')

ks_n_stat, ks_n_p = kstest(group_a, 'norm', args=(group_a.mean(), group_a.std()))
print(f'KS vs Normal:      D={ks_n_stat:.4f}, p={ks_n_p:.4f}')

# Two-sample KS: test if two samples come from same distribution
ks2_stat, ks2_p = ks_2samp(group_a, group_b)
print(f'\n2-sample KS: D={ks2_stat:.4f}, p={ks2_p:.4f}')

## 7. Kernel Density Estimation

In [ ]:
from scipy.stats import gaussian_kde

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# KDE with different bandwidths
x_eval = np.linspace(group_a.min()-1, group_a.max()+1, 300)
for bw, ls in [(0.2,'--'), (0.5,'-'), (1.0,':')]:
    kde = gaussian_kde(group_a, bw_method=bw)
    axes[0].plot(x_eval, kde(x_eval), ls=ls, lw=2, label=f'bw={bw}')
axes[0].hist(group_a, bins=15, density=True, alpha=.3, color='grey')
axes[0].set_title('KDE: Effect of Bandwidth'); axes[0].legend()

# Compare distributions
for arr, lbl, col in [(group_a,'Group A','blue'), (group_b,'Group B','red')]:
    kde = gaussian_kde(arr)
    x_e = np.linspace(0, max(arr.max(),group_b.max())+1, 300)
    axes[1].plot(x_e, kde(x_e), lw=2, color=col, label=lbl)
    axes[1].hist(arr, bins=15, density=True, alpha=.2, color=col)
axes[1].set_title('KDE Comparison: Group A vs B'); axes[1].legend()

plt.tight_layout()
os.makedirs('08_nonparametric_methods', exist_ok=True)
plt.savefig('08_nonparametric_methods/kde_plots.png', dpi=100, bbox_inches='tight')
plt.show(); print('Saved.')

## 8. Bootstrap Confidence Intervals

In [ ]:
# Bootstrap CI for median of group_a
n_boot = 5000
boot_medians = np.array([np.median(resample(group_a)) for _ in range(n_boot)])

ci_low  = np.percentile(boot_medians, 2.5)
ci_high = np.percentile(boot_medians, 97.5)
print(f'Sample median = {np.median(group_a):.4f}')
print(f'Bootstrap 95% CI: ({ci_low:.4f}, {ci_high:.4f})')
print(f'Bootstrap SE = {boot_medians.std():.4f}')

# Bootstrap CI for correlation
boot_rhos = np.array([spearmanr(resample(x_rank, y_rank, random_state=i)[:2]).statistic
                       for i in range(n_boot)])
ci_rho = np.percentile(boot_rhos, [2.5, 97.5])
print(f'\nSpearman rho = {rho:.4f}')
print(f'Bootstrap 95% CI: ({ci_rho[0]:.4f}, {ci_rho[1]:.4f})')

## Key Takeaways

- Use nonparametric tests when normality is violated or sample sizes are small
- Mann-Whitney: non-parametric alternative to two-sample t-test
- Wilcoxon signed-rank: non-parametric alternative to paired t-test
- Kruskal-Wallis: non-parametric alternative to one-way ANOVA
- Bootstrap: flexible, assumption-free inference for any statistic
